In [1]:
import pandas as pd
import numpy as np
import os
import fileinput
import sys

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell

# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
def transfer_to_all_sources(origin, destination, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc'):
    run_dir_in = os.path.join(data_dir, origin, 'run', 'L35n2160TNG')
    run_dir_out = os.path.join(data_dir, destination, 'run', 'L35n2160TNG')
    run_dir_out_input = os.path.join(data_dir, destination, 'input', 'L35n2160TNG')
    snap_dirs = ['sn013', 'sn008', 'sn004']
    for snap in snap_dirs:
        snap_dir_in = os.path.join(run_dir_in, snap)
        snap_dir_out = os.path.join(run_dir_out, snap)
        
        snap_dir_out_input = os.path.join(run_dir_out_input, snap)
        for halo in os.listdir(snap_dir_in):
            halo_dir_in = os.path.join(snap_dir_in, halo)
            halo_dir_out = os.path.join(snap_dir_out, halo)
            
            output_dir_in = os.path.join(halo_dir_in, 'Output')
            
            halo_dir_out_input = os.path.join(snap_dir_out_input, halo)

            config_dir_out = os.path.join(halo_dir_out, 'config')
            last_rt_in = os.path.join(output_dir_in, 'phys_ic00_rt05.out')
            if os.path.isfile(last_rt_in):
                print(f'working on halo {halo} in snap {snap}')
                if not os.path.isdir(halo_dir_out):
                    os.system(f'cp -r {halo_dir_in} {snap_dir_out}')
                else:
                    print(f'{halo} already exists')

In [3]:
def correct_fesc_file(origin, destination, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc', from_freya=True):
    fesc_file = os.path.join(data_dir, destination, '00_f_esc.txt')
    for line in fileinput.input(fesc_file, inplace=1):
        line = line.replace(origin, destination)
        sys.stdout.write(line)
    fileinput.close()
    return

In [4]:
def correct_paths(origin, destination, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc'):
    run_dir_out = os.path.join(data_dir, destination, 'run', 'L35n2160TNG')
    run_dir_out_input = os.path.join(data_dir, destination, 'input', 'L35n2160TNG')
    
    init_path = os.path.join(destination, 'bin', 'CRASH_5.4.3v09.in')
    modules_path = os.path.join(destination, 'config', 'Modules')
    random_path = os.path.join(destination, 'config', 'SYS_RANDOM.in')
    snap_dirs = ['sn013', 'sn008', 'sn004']
    for snap in snap_dirs:
        snap_dir_out = os.path.join(run_dir_out, snap)
        
        snap_dir_out_input = os.path.join(run_dir_out_input, snap)
        for halo in os.listdir(snap_dir_out):
            print(f'Working on {halo} in snap {snap}')
            halo_dir_out = os.path.join(snap_dir_out, halo)
            
            
            halo_dir_out_input = os.path.join(snap_dir_out_input, halo)

            config_dir_out = os.path.join(halo_dir_out, 'config')
            
            simul_file = os.path.join(config_dir_out, 'SIMULATION.in')
            
            for line in fileinput.input(simul_file, inplace=1):
                if (line[:8] == "'IN_DIR=") or (line[:9] == "'OUT_DIR="):
                    line = line.replace(origin, destination)
                sys.stdout.write(line)
            fileinput.close()
            
            input_dir = os.path.join(halo_dir_out, 'Input')
            old_init = os.path.join(halo_dir_out, 'SYS_INIT.in')
            old_modules = os.path.join(halo_dir_out, 'config', 'Modules')
            old_random = os.path.join(halo_dir_out, 'config', 'SYS_RANDOM.in')
            
            source_file = os.path.join(input_dir, 'sources_ic00.in')
            simul_file = os.path.join(input_dir, 'simul_ic00.in')
            dens_file = os.path.join(input_dir, 'dens_ic00.in')
            ion_file = os.path.join(input_dir, 'ion.in')
            temp_file = os.path.join(input_dir, 'temp_ic00.in')
            
            source_file_sources = os.path.join(halo_dir_out_input, 'sources_ic00.in')
            dens_file_sources = os.path.join(halo_dir_out_input, 'dens_ic00.in')
            simul_file_sources = os.path.join(halo_dir_out_input, 'simul_ic00.in')
            ion_file_sources = os.path.join(halo_dir_out_input, 'ion.in')
            temp_file_sources = os.path.join(halo_dir_out_input, 'temp_ic00.in')
            
            os.system(f'rm {source_file}')
            os.system(f'rm {simul_file}')
            os.system(f'rm {dens_file}')
            os.system(f'rm {ion_file}')
            os.system(f'rm {temp_file}')
            os.system(f'rm {old_init}')
            os.system(f'rm {old_modules}')
            os.system(f'rm {old_random}')
            
            os.system(f'ln -s {source_file_sources} {input_dir}')
            os.system(f'ln -s {dens_file_sources} {input_dir}')
            os.system(f'ln -s {simul_file_sources} {input_dir}')
            os.system(f'ln -s {ion_file_sources} {input_dir}')
            os.system(f'ln -s {temp_file_sources} {input_dir}')
            os.system(f'ln -s {init_path} {old_init}')
            os.system(f'ln -s {modules_path} {old_modules}')
            os.system(f'ln -s {random_path} {old_random}')

In [5]:
origin = 'new_main'
destination = 'esc_analysis'
transfer_to_all_sources(origin=origin, destination=destination)

working on halo g18024 in snap sn013
working on halo g5025 in snap sn013
working on halo g36099 in snap sn013
g36099 already exists
working on halo g5444 in snap sn013
g5444 already exists
working on halo g7085 in snap sn013
working on halo g5265 in snap sn013
working on halo g6715 in snap sn013
g6715 already exists
working on halo g6955 in snap sn013
g6955 already exists
working on halo g14147 in snap sn013
g14147 already exists
working on halo g2324 in snap sn013
working on halo g10928 in snap sn013
g10928 already exists
working on halo g34620 in snap sn013
g34620 already exists
working on halo g41356 in snap sn013
g41356 already exists
working on halo g2456 in snap sn013
working on halo g13062 in snap sn013
g13062 already exists
working on halo g24614 in snap sn013
g24614 already exists
working on halo g22417 in snap sn013
g22417 already exists
working on halo g29181 in snap sn013
working on halo g35962 in snap sn013
g35962 already exists
working on halo g11514 in snap sn013
g11514 

working on halo g36987 in snap sn013
g36987 already exists
working on halo g13531 in snap sn013
g13531 already exists
working on halo g5286 in snap sn013
g5286 already exists
working on halo g4552 in snap sn013
working on halo g9716 in snap sn013
g9716 already exists
working on halo g26944 in snap sn013
working on halo g15731 in snap sn013
g15731 already exists
working on halo g26762 in snap sn013
g26762 already exists
working on halo g68590 in snap sn013
g68590 already exists
working on halo g2063 in snap sn013
working on halo g1198 in snap sn013
working on halo g2910 in snap sn013
working on halo g9096 in snap sn013
g9096 already exists
working on halo g11902 in snap sn013
g11902 already exists
working on halo g7021 in snap sn013
working on halo g19754 in snap sn013
g19754 already exists
working on halo g3466 in snap sn013
working on halo g5604 in snap sn013
working on halo g8072 in snap sn013
working on halo g3982 in snap sn013
working on halo g3198 in snap sn013
working on halo g73

working on halo g5535 in snap sn013
g5535 already exists
working on halo g8906 in snap sn013
g8906 already exists
working on halo g2265 in snap sn013
g2265 already exists
working on halo g5419 in snap sn013
working on halo g5941 in snap sn013
g5941 already exists
working on halo g49195 in snap sn013
g49195 already exists
working on halo g3815 in snap sn013
working on halo g27241 in snap sn013
g27241 already exists
working on halo g4171 in snap sn013
g4171 already exists
working on halo g2656 in snap sn013
working on halo g3553 in snap sn013
working on halo g33512 in snap sn013
g33512 already exists
working on halo g4433 in snap sn013
g4433 already exists
working on halo g7299 in snap sn013
working on halo g12303 in snap sn013
g12303 already exists
working on halo g26005 in snap sn013
g26005 already exists
working on halo g2264 in snap sn013
working on halo g19065 in snap sn013
g19065 already exists
working on halo g23628 in snap sn013
g23628 already exists
working on halo g24934 in sna

working on halo g6853 in snap sn013
working on halo g8178 in snap sn013
g8178 already exists
working on halo g16577 in snap sn013
g16577 already exists
working on halo g6676 in snap sn013
g6676 already exists
working on halo g17894 in snap sn013
working on halo g18615 in snap sn013
g18615 already exists
working on halo g5644 in snap sn013
working on halo g46122 in snap sn013
g46122 already exists
working on halo g5031 in snap sn013
working on halo g49796 in snap sn013
g49796 already exists
working on halo g13983 in snap sn013
g13983 already exists
working on halo g36126 in snap sn013
g36126 already exists
working on halo g9557 in snap sn013
g9557 already exists
working on halo g5395 in snap sn013
working on halo g3036 in snap sn013
working on halo g5923 in snap sn013
g5923 already exists
working on halo g4208 in snap sn013
working on halo g15701 in snap sn013
g15701 already exists
working on halo g10160 in snap sn013
g10160 already exists
working on halo g48801 in snap sn013
g48801 alr

working on halo g2582 in snap sn013
working on halo g25620 in snap sn013
g25620 already exists
working on halo g7782 in snap sn013
g7782 already exists
working on halo g7240 in snap sn013
working on halo g34336 in snap sn013
g34336 already exists
working on halo g36953 in snap sn013
g36953 already exists
working on halo g30140 in snap sn013
working on halo g23644 in snap sn013
g23644 already exists
working on halo g1180 in snap sn013
working on halo g43146 in snap sn013
g43146 already exists
working on halo g44696 in snap sn013
g44696 already exists
working on halo g4149 in snap sn013
working on halo g17085 in snap sn013
g17085 already exists
working on halo g16438 in snap sn013
g16438 already exists
working on halo g3988 in snap sn013
g3988 already exists
working on halo g35636 in snap sn013
g35636 already exists
working on halo g9140 in snap sn013
working on halo g2829 in snap sn013
working on halo g6901 in snap sn013
g6901 already exists
working on halo g21945 in snap sn013
g21945 a

...

working on halo g3701 in snap sn004
g3701 already exists
working on halo g1053 in snap sn004
g1053 already exists
working on halo g1176 in snap sn004
g1176 already exists
working on halo g2996 in snap sn004
g2996 already exists
working on halo g477 in snap sn004
g477 already exists
working on halo g1600 in snap sn004
g1600 already exists
working on halo g342 in snap sn004
g342 already exists
working on halo g1088 in snap sn004
g1088 already exists
working on halo g8014 in snap sn004
g8014 already exists
working on halo g901 in snap sn004
g901 already exists
working on halo g1418 in snap sn004
g1418 already exists
working on halo g7277 in snap sn004
g7277 already exists
working on halo g5914 in snap sn004
g5914 already exists
working on halo g437 in snap sn004
working on halo g302 in snap sn004
g302 already exists
working on halo g8616 in snap sn004
g8616 already exists
working on halo g4707 in snap sn004
g4707 already exists
working on halo g3863 in snap sn004
g3863 already exists
work

working on halo g8807 in snap sn004
g8807 already exists
working on halo g1046 in snap sn004
g1046 already exists
working on halo g259 in snap sn004
working on halo g18276 in snap sn004
g18276 already exists
working on halo g2866 in snap sn004
g2866 already exists
working on halo g2012 in snap sn004
g2012 already exists
working on halo g124 in snap sn004
g124 already exists
working on halo g5147 in snap sn004
g5147 already exists
working on halo g5749 in snap sn004
g5749 already exists
working on halo g11923 in snap sn004
g11923 already exists
working on halo g4394 in snap sn004
g4394 already exists
working on halo g171 in snap sn004
g171 already exists
working on halo g1468 in snap sn004
g1468 already exists
working on halo g1274 in snap sn004
g1274 already exists
working on halo g4038 in snap sn004
g4038 already exists
working on halo g1582 in snap sn004
g1582 already exists
working on halo g5243 in snap sn004
g5243 already exists
working on halo g2424 in snap sn004
g2424 already exi

working on halo g1807 in snap sn004
g1807 already exists
working on halo g2017 in snap sn004
g2017 already exists
working on halo g1688 in snap sn004
g1688 already exists
working on halo g1306 in snap sn004
g1306 already exists
working on halo g4371 in snap sn004
g4371 already exists
working on halo g2616 in snap sn004
g2616 already exists
working on halo g2291 in snap sn004
g2291 already exists
working on halo g746 in snap sn004
g746 already exists
working on halo g1504 in snap sn004
g1504 already exists
working on halo g611 in snap sn004
g611 already exists
working on halo g72 in snap sn004
working on halo g4697 in snap sn004
g4697 already exists
working on halo g4098 in snap sn004
g4098 already exists
working on halo g2820 in snap sn004
g2820 already exists
working on halo g1627 in snap sn004
g1627 already exists
working on halo g8636 in snap sn004
g8636 already exists
working on halo g47646 in snap sn004
g47646 already exists
working on halo g6625 in snap sn004
g6625 already exists

working on halo g5341 in snap sn004
g5341 already exists
working on halo g853 in snap sn004
g853 already exists
working on halo g3573 in snap sn004
g3573 already exists
working on halo g1083 in snap sn004
g1083 already exists
working on halo g4132 in snap sn004
g4132 already exists
working on halo g389 in snap sn004
g389 already exists
working on halo g4732 in snap sn004
g4732 already exists
working on halo g8638 in snap sn004
g8638 already exists
working on halo g254 in snap sn004
working on halo g1439 in snap sn004
g1439 already exists
working on halo g2903 in snap sn004
g2903 already exists
working on halo g3192 in snap sn004
g3192 already exists
working on halo g948 in snap sn004
g948 already exists
working on halo g3050 in snap sn004
g3050 already exists
working on halo g813 in snap sn004
g813 already exists
working on halo g7187 in snap sn004
g7187 already exists
working on halo g627 in snap sn004
g627 already exists
working on halo g3052 in snap sn004
g3052 already exists
workin

working on halo g1001 in snap sn004
working on halo g1235 in snap sn004
g1235 already exists
working on halo g3667 in snap sn004
g3667 already exists
working on halo g1475 in snap sn004
g1475 already exists
working on halo g3010 in snap sn004
g3010 already exists
working on halo g2847 in snap sn004
g2847 already exists
working on halo g2260 in snap sn004
g2260 already exists
working on halo g241 in snap sn004
g241 already exists
working on halo g5374 in snap sn004
g5374 already exists
working on halo g800 in snap sn004
g800 already exists
working on halo g4613 in snap sn004
g4613 already exists
working on halo g336 in snap sn004
g336 already exists
working on halo g201 in snap sn004
g201 already exists
working on halo g1896 in snap sn004
g1896 already exists
working on halo g2371 in snap sn004
g2371 already exists
working on halo g661 in snap sn004
g661 already exists
working on halo g77 in snap sn004
g77 already exists
working on halo g197 in snap sn004
g197 already exists
working on 

In [ ]:
origin = 'new_main'
destination = 'esc_analysis'
correct_fesc_files(origin=origin, destination=destination)

In [ ]:
origin = 'new_main'
destination = 'esc_analysis'
correct_paths(origin=origin, destination=destination)

In [52]:
origin = '/raven/ptmp/ivkos/new_5e-1'
destination = '/ptmp/mpa/mglatzle/TNG_f_esc/new_5e-1'
correct_fesc_file(origin, destination, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc', from_freya=True)

In [53]:
origin = '/raven/ptmp/ivkos/new_5e-1'
destination = '/ptmp/mpa/mglatzle/TNG_f_esc/new_5e-1'
correct_paths(origin=origin, destination=destination)

Working on g29181 in snap sn013
Working on g61 in snap sn013
Working on g13697 in snap sn013
Working on g48185 in snap sn013
Working on g13632 in snap sn013
Working on g936 in snap sn013
Working on g202 in snap sn013
Working on g21316 in snap sn013
Working on g4096 in snap sn013
Working on g13393 in snap sn013
Working on g41 in snap sn013
Working on g11902 in snap sn013
Working on g15689 in snap sn013
Working on g823 in snap sn013
Working on g502 in snap sn013
Working on g3538 in snap sn013
Working on g184 in snap sn013
Working on g3125 in snap sn013
Working on g3081 in snap sn013
Working on g838 in snap sn013
Working on g2716 in snap sn013
Working on g14510 in snap sn013
Working on g250 in snap sn013
Working on g159 in snap sn013
Working on g14152 in snap sn013
Working on g15825 in snap sn013
Working on g5535 in snap sn013
Working on g404 in snap sn013
Working on g3553 in snap sn013
Working on g130 in snap sn013
Working on g34927 in snap sn013
Working on g64873 in snap sn013
Working o

Working on g1099 in snap sn013
Working on g10108 in snap sn013
Working on g1420 in snap sn013
Working on g1159 in snap sn013
Working on g24903 in snap sn013
Working on g1801 in snap sn013
Working on g269 in snap sn013
Working on g30778 in snap sn013
Working on g4498 in snap sn013
Working on g280 in snap sn013
Working on g13121 in snap sn013
Working on g1544 in snap sn013
Working on g2322 in snap sn013
Working on g35658 in snap sn013
Working on g27145 in snap sn013
Working on g59 in snap sn013
Working on g160 in snap sn013
Working on g1619 in snap sn013
Working on g20126 in snap sn013
Working on g960 in snap sn013
Working on g47 in snap sn013
Working on g869 in snap sn013
Working on g89 in snap sn013
Working on g10485 in snap sn013
Working on g6404 in snap sn013
Working on g18435 in snap sn013
Working on g28453 in snap sn013
Working on g982 in snap sn013
Working on g32874 in snap sn013
Working on g3959 in snap sn013
Working on g482 in snap sn013
Working on g10984 in snap sn013
Working o

Working on g448 in snap sn008
Working on g2570 in snap sn008
Working on g360 in snap sn008
Working on g13092 in snap sn008
Working on g12909 in snap sn008
Working on g328 in snap sn008
Working on g240 in snap sn008
Working on g149 in snap sn008
Working on g2499 in snap sn008
Working on g59 in snap sn008
Working on g9516 in snap sn008
Working on g120 in snap sn008
Working on g17317 in snap sn008
Working on g1118 in snap sn008
Working on g449 in snap sn008
Working on g314 in snap sn008
Working on g960 in snap sn008
Working on g3559 in snap sn008
Working on g13228 in snap sn008
Working on g5136 in snap sn008
Working on g35 in snap sn008
Working on g3674 in snap sn008
Working on g982 in snap sn008
Working on g1896 in snap sn008
Working on g756 in snap sn008
Working on g73 in snap sn008
Working on g435 in snap sn008
Working on g5165 in snap sn008
Working on g23 in snap sn008
Working on g815 in snap sn008
Working on g1146 in snap sn004
Working on g1162 in snap sn004
Working on g65 in snap sn

In [30]:
origin = 'new_main'
destination = 'esc_analysis'
correct_fesc_file(origin=origin, destination=destination)

In [11]:
testpath = '/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g1975/SYS_INIT.in'
print(origin not in os.path.realpath(testpath))

True


In [48]:
def remove_halo(destination, snap, halo, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc'):
    esc_file = os.path.join(data_dir, destination, '00_f_esc.txt')
    file = open(esc_file, "r")
    file_new = open(esc_file+'_new', 'w')
    for line in file:
        if f'{snap}/g{halo}' not in line:
            file_new.write(line)
#         line = line.strip()
#         changes = line.replace("hardships", "situations")
#         replacement = replacement + changes + "\n"

    file.close()
    file_new.close()

In [60]:
broken = [2133, 2459, 1158, 1057, 1667, 121, 1519]

[2133, 2459, 1158, 1057, 1667, 121]

In [95]:
broken008 = [2069, 3819, 1132]

In [62]:
remove_halo('esc_analysis', 'sn004', '1519')

In [56]:
correct_fesc_file(origin, destination, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc', from_freya=True)

In [63]:
from crashpy.dataclasses.simulation import LoadedSimulation as Sim

In [86]:
import pickle

In [88]:
with open('now_working', 'rb') as f:
    work_list = pickle.load(f)

In [89]:
work_list

['/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g3619',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g1132']

In [96]:
path = '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g1293'

In [97]:
csim = Sim(path)